# Retrieval Augmented Generation using Sparse retrieval using Elastic, Anthropic Claude 3.7, Amazon Bedrock and Langchain

## Introduction

In this notebook we will show you how to use Elastic search, Amazon Bedrock, Anthropic Claude 3.7 and Langchain to build a Retrieval Augmented Generation (RAG) solution


#### Use case

To demonstrate the RAG capability, let's take the use case of an AI Assistant that can help answer questions from a personal document. 


#### Persona
You are Bob,an Application Developer at Anycompany. Anycompany is experiencing an overwhelming number of customer queries. Anycompany has built a secure and performant conversational AI Assistant to answer frequently asked questions. Now Anycompany wants this conversational AI assistant to be able to answer questions are which are specific to the company. Also, you would like to retrieve documents using a sparse retrieval mechanism.

In this workshop, you will build a context aware conversational AI Assistant using a sparse retrieval strategy for Anycompany 

#### Implementation
To fulfill this use case, in this notebook we will show how to create a RAG Application using a sparse retrieval  strategy to answer questions from business data. We will use  Elasticsearch, Anthropic Claude 3.7 Sonnet Foundation model, Amazon Bedrock and Langchain. 

#### Prerequisites
We're using an Elastic Serverless deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud account, sign up [here](https://cloud.elastic.co/serverless-registration?onboarding_token=vectorsearch) for a free trial. Once you've signed up, you can create your Elasticsearch API Key following the documentation [here](https://www.elastic.co/docs/current/serverless/api-keys). 

In additon, make sure to go to **Trained Models** page under **Project Settings** (or **Management** if you haven't set up an Elasticsearch project) on left menu bar and then **Download** *elser* model.

![elser-model-download.png](static/elser-model-download.png)



#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore, elasticsearch and langchain.

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_aws --force-reinstall --quiet
%pip install langchain-elasticsearch --force-reinstall --quiet
%pip install elasticsearch==8.18.0 --force-reinstall --quiet
%pip install unstructured==0.7.12 --force-reinstall --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.13 requires botocore==1.34.131, but you have botocore 1.35.76 which is incompatible.
sagemaker 2.224.1 requires attrs<24,>=23.1.0, but you have attrs 24.2.0 which is incompatible.
sagemaker 2.224.1 requires protobuf<5.0,>=3.12, but you have protobuf 5.29.1 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.13 requires botocore==1.34.131, but you have botocore 1.35.76 which is incompatible.
sagemaker 2.224.1 requires

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup 

Import the necessary libraries

In [4]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrockConverse
from langchain_aws import AmazonKnowledgeBasesRetriever
from langchain_aws import BedrockEmbeddings
from botocore.client import Config
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch, exceptions
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQA
from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from pathlib import Path

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [ ]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0' # change this to use a different version from the model provider
embeddingmodelId = 'amazon.titan-embed-text-v2:0' # change this to use a different embedding model

llm = ChatBedrockConverse(model_id=modelId, client=bedrock_client)

## Read files from directory

Load all PDF files which are present in the directory

In [6]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [7]:
TMP_DIR = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'media')
loader = DirectoryLoader(TMP_DIR, glob='**/*.pdf')
documents = loader.load()

## Split Documents

Chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 1000 token passages with an overlap of 0 tokens.

Here we are using Recursive Character Text splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000,
        chunk_overlap=0
        )
texts = text_splitter.split_documents(documents)

## Connect to Elasticsearch

We'll use the Cloud ID to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to [Cloud ID](https://cloud.elastic.co/deployments) and select your deployment.

We will use ElasticsearchStore to connect to our elastic cloud deployment. This would help create and index data easily. 

In [9]:
cloud_id = getpass("Elastic deployment Cloud ID: ")
cloud_username = "elastic"
cloud_api_key = getpass("Elastic deployment API Key: ")
index_name= "new-index-1"

vector_store = ElasticsearchStore(
        es_cloud_id=cloud_id,  
        index_name= index_name,
        es_api_key=cloud_api_key)

Elastic deployment Cloud ID:  ········
Elastic deployment API Key:  ········


## Index data into Elasticsearch and initialize retriever for sparse retrieval

Next, we will index data to elasticsearch using ElasticsearchStore.from_documents. We will use Cloud ID, Password and Index name values set in the Create cloud deployment step.

Elasticsearch has the ability to support a wide range of retrieval strategies. By default, ElasticsearchStore uses the ApproxRetrievalStrategy

This below code block will show how to configure ElasticsearchStore to perform Elasticsearch’s sparse vector retrieval to retrieve the top-k results. Elastic only support its own “ELSER” embedding model for now.

NOTE This requires the ELSER model to be deployed and running. The code block below will take care of this; if you want to do this from Kibana in Elastic directly, you can follow the instructions here: [Download and deploy ELSER](https://www.elastic.co/guide/en/machine-learning/master/ml-nlp-elser.html#download-deploy-elser). 

In [10]:
model_id = ".elser_model_2_linux-x86_64"

# create Elasticsearch client
client = Elasticsearch(
        cloud_id=cloud_id,  
        api_key=cloud_api_key)

try:
    response = client.ml.start_trained_model_deployment(model_id=model_id)
    print(f"Deployment status: {response}")

except exceptions.BadRequestError as e:
    if e.error == "status_exception":
        print("'Could not start model deployment because an existing deployment with the same id [.elser_model_2_linux-x86_64] exists already. You're good to go.")
    else:
        raise e

'Could not start model deployment because an existing deployment with the same id [.elser_model_2_linux-x86_64] exists already. You're good to go.


In [11]:
vectordb = vector_store.from_documents(
        texts,
        index_name=index_name,
        es_cloud_id=cloud_id,
        es_api_key=cloud_api_key,
        strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=".elser_model_2_linux-x86_64")
        )

retriever = vectordb.as_retriever()

## Model Invocation and Response Generation using RetrievalQA chain

Now that we have the passages stored in Elasticsearch and LLM is initialized, we can now ask a question to get the relevant passages.

In [12]:
query = "What is an important first step for machine learning metamodeling?"


machinelearning_advisor_template = """
    Human: You will be acting as a Machine Learning advisor on complex Mechanical systems named Poly created by the company Polymath. 
    Your goal is to give advice related to Application of Machine Learning on Mechanical Systems to users. You will be replying to users 
    who are asking questions on Application of Machine Learning in Mechanical Systems 
    site and who will be confused if you don't respond in the character of Poly.
    
    You should maintain a friendly customer service tone.

    Here is the document you should reference when answering the user: <context>{context}</context>

    Here are some important rules for the interaction:
    - Always stay in character, as Poly, a Machine Learning advisor on complex Mechanical systems
    - If you are unsure how to respond, say “Sorry, I didn’t understand that. Could you repeat the question?”
    - If someone asks something irrelevant, say, “Sorry, I am Poly and I give career advice. Do you have a question related to Application of Machine Learning in Mechanical Systems today I can help you with?”

    Here is an example of how to respond in a standard interaction:

    <example>
    User: Hi, how were you created and what do you do?
    Poly: Hello! My name is Poly, and I was created by Polymath  to give advice on Application of Machine Learning in Mechanical Systems. 
        What can I help you with today?
    User: Hi, how can I use Decision Trees?
    Poly: The method for using decision trees described in the given context is to first rank all
        all the rules within the decision tree to find the most meaningful and reliable design subspaces.
        Then, determine how many significant rules exist within the dataset to present the engineer
        with the most reliable and important knowledge. Finally, use the resulting rules from the decision tree
        to make predictions or decisions based on the given target variable.
        User: Hi, What is Machine Learning?
    Poly: Machine learning is a tool that allows predictions about future behavior to be drawn from existing
        data sets. It is used in everything from spam filters to self-driving cars. While the field has been
        around for decades, it has flourished over the last several years as computing power has increased
        and user-friendly toolkits have been developed in a variety of programming languages. We do not
        go into detail on machine learning theory and practice in this paper, but we do introduce the
        concepts necessary for understanding our implementation of it.
    </example>

    Here is the user’s question: <question> {question} </question>

    How do you respond to the user’s question?
    Think about your answer first before you respond. Put your response in <response></response> tags.
    Assistant: <response>"""

prompt = PromptTemplate(template=machinelearning_advisor_template, input_variables=["context","question"])
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": prompt})
response = qa_chain.invoke(query)
print(response["result"])

/opt/conda/lib/python3.11/site-packages/langchain_elasticsearch/vectorstores.py:868: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  hits = self._store.search(


According to the context provided, framing the problem is the most important first step for machine learning metamodeling. The context states:

"The first step—framing the problem—may be the most important. Before you train a machine learning algorithm, you must determine what kind of question you are asking, what kind of data you need to answer it, and how you are going to collect that data."

Framing the problem involves defining the question you want to answer, identifying the data required, and planning how to collect that data. This crucial initial step sets the foundation for effective machine learning metamodeling and ensures the right approach is taken.

</response>


## Delete Elasticsearch Index

Delete the Elasticsearch index

In [13]:
es = Elasticsearch(cloud_id=cloud_id, api_key=cloud_api_key)
es.options(ignore_status=[400,404]).indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})

## Conclusion
You have now experimented with `langchain` SDK to get an exposure to RAG Application building using Elastic, Anthropic Claude 3.7 and Amazon Bedrock API. Using langchain you have asked questions on your domain specicific documents.

### Take aways
- Adapt this notebook to experiment with different Claude 3 models available through Amazon Bedrock. 
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You